# 构建聊天机器人

:::note

本教程之前使用了 [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) 抽象。您可以在 [v0.2 文档](https://python.langchain.com/v0.2/docs/tutorials/chatbot/) 中找到该版本的文档。

自 LangChain v0.3 版本发布以来，我们建议 LangChain 用户利用 [LangGraph 持久化](https://langchain-ai.github.io/langgraph/concepts/persistence/) 来将 `memory` 整合到新的 LangChain 应用程序中。

如果您的代码已在使用 `RunnableWithMessageHistory` 或 `BaseChatMessageHistory`，则**无需**进行任何更改。我们不打算在不久的将来弃用此功能，因为它适用于简单的聊天应用程序，并且任何使用 `RunnableWithMessageHistory` 的代码将继续按预期工作。

请参阅[如何迁移到 LangGraph Memory](/docs/versions/migrating_memory/) 以获取更多详细信息。
:::

## 概述

我们将通过一个示例来介绍如何设计和实现一个由 LLM 驱动的聊天机器人。
这个聊天机器人将能够进行对话并记住与 [chat model](/docs/concepts/chat_models) 之前的交互。


请注意，我们将构建的这个聊天机器人将仅使用语言模型进行对话。
有几个相关的概念您可能正在寻找：

- [面向对话型的 RAG](/docs/tutorials/qa_chat_history)：为外部数据源提供聊天机器人体验
- [Agents](/docs/tutorials/agents)：构建一个能够执行操作的聊天机器人

本教程将涵盖基础知识，这将有助于您学习这两个更高级的主题，但如果您愿意，也可以直接跳到那里。

## 设置

### Jupyter Notebook

本指南（以及文档中的大部分其他指南）使用了 [Jupyter notebooks](https://jupyter.org/)，并假定读者也在使用它。Jupyter notebooks 非常适合学习如何使用 LLM 系统，因为事情常常会出错（意外输出、API 宕机等），而在交互式环境中学习指南是更好地理解它们的绝佳方式。

本教程及其他教程在使用 Jupyter notebook 时可能最为方便。请参阅[此处](https://jupyter.org/install)了解安装说明。

### 安装

在本教程中，我们将需要 `langchain-core` 和 `langgraph`。本指南需要 `langgraph >= 0.2.28`。

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain-core langgraph>0.2.27</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain-core langgraph>0.2.27 -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>



有关更多详细信息，请参阅我们的[安装指南](/docs/how_to/installation)。

### LangSmith

您使用 LangChain 构建的许多应用程序将包含多个步骤和多次调用 LLM。
随着这些应用程序变得越来越复杂，能够检查链或代理内部到底发生了什么变得至关重要。
做到这一点的最佳方法是使用 [LangSmith](https://smith.langchain.com)。

在上面的链接处注册后，请确保设置您的环境变量以开始记录跟踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

或者，如果您在 notebook 中，可以使用以下方式设置：

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

## 快速入门

首先，让我们学习如何单独使用语言模型。LangChain 支持许多不同的语言模型，您可以互换使用——选择您想要使用的模型！

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4o-mini"}}} />

In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

让我们先直接使用模型。`ChatModel` 是 LangChain "Runnables" 的实例，这意味着它们公开了一个标准的交互接口。要直接调用模型，我们可以将消息列表传递给 `.invoke` 方法。

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-5211544f-da9f-4325-8b8e-b3d92b2fc71a-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

模型本身不具备任何状态概念。例如，如果你提出一个后续问题：

In [4]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't have access to personal information about users unless it has been shared with me in the course of our conversation. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 11, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2d13a18-7022-4784-b54f-f85c097d1075-0', usage_metadata={'input_tokens': 11, 'output_tokens': 34, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

让我们看看这个 [LangSmith 追踪示例](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r)

我们可以看到它没有将之前的对话内容纳入上下文，并且无法回答问题。
这导致了糟糕的聊天机器人体验！

为了解决这个问题，我们需要将整个[对话历史](/docs/concepts/chat_history) 传递给模型。让我们看看这样做会发生什么：

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 33, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-34bcccb3-446e-42f2-b1de-52c09936c02c-0', usage_metadata={'input_tokens': 33, 'output_tokens': 14, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

现在我们可以看到，我们得到了一个很好的响应！

这就是支撑聊天机器人进行对话式交互能力的基本原理。那么，我们如何最好地实现这一点呢？

## 消息持久化

[LangGraph](https://langchain-ai.github.io/langgraph/) 实现了一个内置的持久化层，这使得它非常适合支持多轮对话的聊天应用程序。

将我们的聊天模型包装在一个最小化的 LangGraph 应用程序中，可以让我们自动持久化消息历史记录，从而简化多轮应用程序的开发。

LangGraph 带有一个简单的内存检查点，我们在下面使用它。有关更多详细信息，包括如何使用不同的持久化后端（例如 SQLite 或 Postgres），请参阅其[文档](https://langchain-ai.github.io/langgraph/concepts/persistence/)。

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

我们现在需要创建一个 `config`，每次都将其传递给 runnable。此配置包含的信息不直接属于输入，但仍然非常有用。在这种情况下，我们希望包含一个 `thread_id`。它应该看起来像这样：

In [7]:
config = {"configurable": {"thread_id": "abc123"}}

这使我们能够通过单个应用程序支持多个会话线程，当您的应用程序拥有多个用户时，这是一个常见需求。

然后，我们可以调用该应用程序：

In [8]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [9]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! How can I help you today, Bob?


太棒了！我们的聊天机器人现在能够记住我们的信息了。如果我们更改配置以引用不同的 `thread_id`，我们就能看到它会重新开始对话。

In [10]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about you unless you've shared it in this conversation. How can I assist you today?


但是，我们总是可以回顾最初的对话（因为我们将其保存在数据库中）

In [11]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. What would you like to discuss today?


这就是我们如何支持一个能够与多位用户进行对话的聊天机器人！

:::tip

为了实现异步支持，请将 `call_model` 节点更新为一个异步函数，并在调用应用程序时使用 `.ainvoke`：

```python
# 异步函数（用于节点）：
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# 像之前一样定义图：
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# 异步调用：
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()
```

:::

目前，我们所做的只是在模型周围添加了一个简单的持久化层。通过加入 Prompt 模板，我们可以开始让聊天机器人变得更加复杂和个性化。

## Prompt 模板

[Prompt 模板](/docs/concepts/prompt_templates) 有助于将原始用户信息转化为 LLM 可以处理的格式。在这种情况下，原始用户输入只是一个消息，我们将其传递给 LLM。现在让我们稍微复杂化一点。首先，我们将添加一个具有一些自定义指令的系统消息（但仍以消息作为输入）。接下来，我们将添加除了消息之外的更多输入。

为了添加系统消息，我们将创建一个 `ChatPromptTemplate`。我们将利用 `MessagesPlaceholder` 来传递所有消息。

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

现在，我们可以更新我们的应用程序以整合这个模板：

In [13]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    # highlight-start
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    # highlight-end
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

我们以同样的方式调用应用程序：

In [14]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! What brings ye to these waters today? Be ye seekin' treasure, knowledge, or perhaps a good tale from the high seas? Arrr!


In [15]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ye be called Jim, matey! A fine name fer a swashbuckler such as yerself! What else can I do fer ye? Arrr!


太棒了！现在让我们把提示词变得稍微复杂一点。假设提示词模板现在看起来是这样的：

In [19]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

请注意，我们已在 prompt 中添加了一个新的 `language` 输入。我们的应用程序现在有两个参数——输入 `messages` 和 `language`。我们应该更新我们应用程序的状态以反映这一点：

In [20]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


# highlight-next-line
class State(TypedDict):
    # highlight-next-line
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # highlight-next-line
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    # highlight-next-line
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿Cómo puedo ayudarte hoy?


请注意，整个状态都会被持久化，因此如果不需要更改参数（例如 `language`），可以省略它们：

In [22]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob. ¿Hay algo más en lo que pueda ayudarte?


为了帮助您了解内部发生的情况，请查看 [此 LangSmith 跟踪](https://smith.langchain.com/public/15bd8589-005c-4812-b9b9-23e74ba4c3c6/r)。

## 管理对话历史

构建聊天机器人时，一个重要的概念是了解如何管理对话历史。如果不加以管理，消息列表将无界增长，并可能溢出 LLM 的上下文窗口。因此，在传递消息时添加一个限制消息大小的步骤非常重要。

**重要的是，您应该在提示模板之前，但在从消息历史记录加载了先前的消息之后完成此操作。**

我们可以通过在提示前面添加一个修改 `messages` 键的简单步骤来实现这一点，然后将该新链包装在 Message History 类中。

LangChain 提供了一些内置的帮助程序来[管理消息列表](/docs/how_to/#messages)。在本例中，我们将使用 [trim_messages](/docs/how_to/trim_messages/) 帮助程序来减少发送给模型的消息数量。修剪器允许我们指定要保留多少个 token，以及其他参数，例如是否始终保留系统消息以及是否允许部分消息：

In [23]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

为了在我们的链中使用它，我们只需要在将 `messages` 输入传递给我们的提示之前运行修剪器。

In [24]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    # highlight-start
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    # highlight-end
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

现在如果我们尝试询问模型我们的名字，它将不知道，因为我们修剪了聊天记录的这部分：

In [25]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

# highlight-next-line
input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. You haven't told me yet!


但是，如果我们询问最近几条消息中的信息，它会记住：

In [26]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked what 2 + 2 equals.


如果你看一下 LangSmith，在 [LangSmith trace](https://smith.langchain.com/public/04402eaa-29e6-4bb1-aa91-885b730b6c21/r) 中，你可以确切地看到幕后发生的一切。

## 流式传输

现在我们有了一个可以工作的聊天机器人。然而，对于聊天机器人应用程序而言，一个*非常*重要的用户体验考量是流式传输。大型语言模型有时响应需要一些时间，因此为了改善用户体验，大多数应用程序会选择在生成每个 token 时就将其流式返回。这可以让用户看到进度。

实际上，做到这一点非常简单！

默认情况下，LangGraph 应用程序中的 `.stream` 会流式传输应用程序的步骤——在本例中是模型响应的单个步骤。将 `stream_mode` 设置为 `"messages"` 允许我们流式传输输出 token：

In [27]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
# highlight-next-line
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    # highlight-next-line
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|’s| a| joke| for| you|:

|Why| don|’t| skeleton|s| fight| each| other|?

|Because| they| don|’t| have| the| guts|!||

## 后续步骤

现在你已经了解了在 LangChain 中创建聊天机器人的基础知识，以下是一些你可能感兴趣的高级教程：

- [对话式 RAG](/docs/tutorials/qa_chat_history): 为外部数据源启用聊天机器人体验
- [Agents](/docs/tutorials/agents): 构建一个能够执行操作的聊天机器人

如果你想深入了解具体细节，可以关注以下内容：

- [流式输出](/docs/how_to/streaming): 流式输出对于聊天应用程序至关重要
- [如何添加消息历史记录](/docs/how_to/message_history): 深入了解所有与消息历史记录相关的内容
- [如何管理大型消息历史记录](/docs/how_to/trim_messages/): 更多管理大型聊天历史记录的技术
- [LangGraph 主文档](https://langchain-ai.github.io/langgraph/): 详细了解如何使用 LangGraph 进行构建